## Cluster MIBI Samples

**Samples:** 
- MIBI FOV Size: 800 μm Frame Size: 2048 (0.39 μm per pixel)

**Method**: Cluster Umap with Leiden algorithm 

Compare to images

compare to hitological subtypes

In [ ]:
#load libraries
import pandas as pd
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import scipy
import scanpy as sc
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
mpl.rc('figure', max_open_warning = 0)
codedir = os.getcwd()
import phenograph
os.chdir('/home/groups/graylab_share/OMERO.rdsStore/engje/Data')
from mplex_image import visualize as viz, process, preprocess, normalize, mics, mpimage
np.random.seed(1027)

In [ ]:
codedir = '/home/groups/graylab_share/OMERO.rdsStore/engje/Data/Keren et al_'

In [ ]:
#change to correct directory
datadir =  f'{codedir}'
s_date = '20220410'
os.chdir(codedir)
if not os.path.exists(s_date):
    os.mkdir(s_date)
%matplotlib inline

In [ ]:
#os.listdir()
#os.listdir('results')
#os.listdir('intermediate_data/protein_positivity')

# Table of contents <a name="contents"></a>
2. [Load annotation](#loadold)
2. [Load intensity](#load)
4. [Normalize](#leiden)
5. [Umap](#l7)
6. [Leiden](#l8)
[annotate](#lbar)
5. [Sub-cluster](#kmeans)
8. [Visualize](#viz)
9. [subtype](#subtype)
9. [survival](#surv)

# Load Annotation <a name="loadold"></a>

[contents](#contents)

In [ ]:
#positve
df_name = pd.read_csv(f'{datadir}/proteins_by_frame.csv',index_col=0)
df_a = pd.read_csv(f'{datadir}/clinical_data.csv',index_col=0)
#load annotation
df_a['Group'] = pd.qcut(x=df_a.loc[:,'Survival_time'],q=2,labels=['short','long'])
d_a = dict(zip(df_a.index.astype('str'),df_a.Group))
d_name = dict(zip(df_name.index.astype('str'),df_name.Biomarker))


# Load Mean Intensity <a name="load"></a>

load data

[contents](#contents)

In [ ]:
df_mi_full = pd.read_csv(f'{codedir}/Segmentation/features_MIBI-TMA_MeanIntensity_Centroid_Shape.csv',index_col=0)

In [ ]:
df_mi_full.columns

In [ ]:
set([item.split('_')[0] + '_cytoplasm' for item in sorted(df_mi_full.columns)])
ls_marker = [#'Au_cytoplasm', 'Background_cytoplasm',
 'Beta catenin_cytoplasm','Beta catenin_nuclei',
 'CD11b_cytoplasm', 'CD11c_cytoplasm', 'CD138_cytoplasm', 'CD16_cytoplasm', 'CD209_cytoplasm', 'CD20_cytoplasm',
 'CD31_cytoplasm', 'CD3_cytoplasm', 'CD45RO_cytoplasm', 'CD45_cytoplasm', 'CD4_cytoplasm', 'CD56_cytoplasm',
 'CD63_cytoplasm', 'CD68_cytoplasm', 'CD8_cytoplasm', #'Ca_cytoplasm','Fe_cytoplasm',
 'EGFR_cytoplasm', 'FoxP3_nuclei', 'H3K27me3_nuclei', 'H3K9ac_nuclei',
 'HLA-Class-1_cytoplasm', 'HLA-DR_cytoplasm', 'IDO_cytoplasm', 'Keratin17_cytoplasm', 'Keratin6_cytoplasm',
 'Ki67_nuclei', 'Lag3_cytoplasm', 'MPO_cytoplasm',# 'Na_cytoplasm',
 'PD-L1_cytoplasm', 'PD1_cytoplasm',# 'P_cytoplasm',
 'Pan-Keratin_cytoplasm', 'SMA_cytoplasm',# 'Si_cytoplasm', 'Ta_cytoplasm',
 'Vimentin_cytoplasm', 'area_nuclei', 'centroid-0_nuclei', 'centroid-1_nuclei', 'eccentricity_nuclei',# 'label_cytoplasm',
 'p53_nuclei', 'phospho-S6_cytoplasm']

In [ ]:
d_replace = {'centroid-0_nuclei':'DAPI_Y',
       'centroid-1_nuclei':'DAPI_X',
            'Beta catenin_cytoplasm':'Beta-catenin-c',
           'Beta catenin_nuclei':'Beta-catenin-n',
            'CD11b_cytoplasm': 'CD11b',
 'CD11c_cytoplasm': 'CD11c',
 'CD138_cytoplasm': 'CD138',
 'CD16_cytoplasm': 'CD16',
 'CD209_cytoplasm': 'CD209',
 'CD20_cytoplasm': 'CD20',
 'CD31_cytoplasm': 'CD31',
 'CD3_cytoplasm': 'CD3',
 'CD45RO_cytoplasm': 'CD45RO',
 'CD45_cytoplasm': 'CD45',
 'CD4_cytoplasm': 'CD4',
 'CD56_cytoplasm': 'CD56',
 'CD63_cytoplasm': 'CD63',
 'CD68_cytoplasm': 'CD68',
 'CD8_cytoplasm': 'CD8',
 'EGFR_cytoplasm': 'EGFR',
 'FoxP3_nuclei': 'FoxP3',
 'H3K27me3_nuclei': 'H3K27me3',
 'H3K9ac_nuclei': 'H3K9ac',
 'HLA-Class-1_cytoplasm': 'HLA-Class-1',
 'HLA-DR_cytoplasm': 'HLA-DR',
 'IDO_cytoplasm': 'IDO',
 'Keratin17_cytoplasm': 'Keratin17',
 'Keratin6_cytoplasm': 'Keratin6',
 'Ki67_nuclei': 'Ki67',
 'Lag3_cytoplasm': 'Lag3',
 'MPO_cytoplasm': 'MPO',
 'PD-L1_cytoplasm': 'PD-L1',
 'PD1_cytoplasm': 'PD1',
 'Pan-Keratin_cytoplasm': 'Pan-Keratin',
 'SMA_cytoplasm': 'SMA',
 'Vimentin_cytoplasm': 'Vimentin',
 'area_nuclei': 'area',
 'eccentricity_nuclei': 'eccentricity',
 'p53_nuclei': 'p53',
 'phospho-S6_cytoplasm': 'phospho-S6'}
df_mi = df_mi_full.loc[:,ls_marker].rename(d_replace,axis=1)
             

In [ ]:
'''['Au',
 'Background',
 'Beta catenin',#good
 'CD11b', #good
 'CD11c',#good
 'CD138', #? CD45 negative
 'CD16', #good
 'CD20', #good
 'CD209',#? rare DC-Sign
 'CD3',#good
 'CD31', #okay
 'CD4', #good
 'CD45',#good
 'CD45RO',#same as CD45
 'CD56',#rare
 'CD63',#good
 'CD68',#good
 'CD8', #good
 'Ca',
 'Celltype',
 'Column',
 'EGFR', #dim, but tumor specific
 'Fe',
 'FoxP3', #rare
 'H3K27me3',#everywhere
 'H3K9ac',
 'HLA-DR',#good
 'HLA_Class_1',#good
 'IDO',#probably okay.
 'Keratin17',#good
 'Keratin6',#good
 'Ki67',#good
 'Lag3',#rare
 'MPO',#rare
 'Na',
 'P',
 'PD-L1',#okay
 'PD1',#good rare
 'Pan-Keratin',#good
 'Row',
 'SMA',#brigt
 'Sample',
 'Si',
 'Ta',
 'Vimentin',#good
 'dsDNA',
 'p53',#good
 'phospho-S6'#okay
]'''


In [ ]:
ls_drop = ['Au', 'Background', 'Ca',  'Fe', 'Na', 'P','Si', 'Ta'] #'Celltype', 'Column','Row','Sample' 

In [ ]:
#df_mi.columns = [item.split('_')[0] for item in df_mi.columns]

# Failed':{'CC3-647','CD4','CD3','F4-80','MHCII','ELN','NFkB','CC3-488', 'panCK','Arg1',}
df_filter_mi = df_mi.loc[:,~df_mi.columns.isin(ls_drop)]
#df_mi.rename({'nuclei_area':'area','nuclei_eccentricity':'eccentricity','slide_scene':'slidescene'},axis=1,inplace=True)
#df_xy.rename({'DAPI_X':'DAPI_X_old','DAPI_Y':'DAPI_Y_old','slide_scene':'slide_scene_old'},axis=1,inplace=True)

In [ ]:
#df_mi.columns = [item.split('_')[0] for item in df_mi.columns]
df_filter_mi.columns

In [ ]:

#s_date
s_sample='MIBI'
s_date

In [ ]:
s_out = f'{codedir}/{s_date}_{s_sample}_FilteredMeanIntensity.csv' #1027 old, 1029 updated
if not os.path.exists(s_out):
    print('saving csv')
    df_filter_mi.to_csv(s_out)

# Normalize <a name="leiden"></a>

background subtraction


[contents](#contents)

In [ ]:
'''
os.chdir(codedir)
s_date = '20211027' #326562 rows
s_date = '20211029' #748728  rows
s_date = '20211109' #722760 rows
s_sample = f'{s_date}_RS-mTMA-5'
df_mi = pd.read_csv(f'{s_sample}_FilteredMeanIntensity.csv',index_col=0)
'''

In [ ]:
#subtract background
'''
d_bg = {'CD31':8,#11,
        'CD45':3, 'CD8':10, 'ColIV':12, 'FoxP3':18,#12,
 'CD103':10, 'CD11b':1.5, 'CD11c':6
}
df_bg = df_mi.copy()
for s_marker, i_thresh in d_bg.items():
    df_bg[s_marker] = np.clip(df_mi.loc[:,s_marker] - i_thresh*256, a_min=0, a_max=None)

s_out = f'{codedir}/{s_sample}_BGSubtractedMeanIntensity.csv'
if not os.path.exists(s_out):
    print('saving csv')
    df_bg.to_csv(s_out)
'''

## Umap <a name="l7"></a>


[contents](#contents)

In [ ]:
#load annotation
df_a['Group'] = pd.qcut(x=df_a.loc[:,'Survival_time'],q=2,labels=['short','long'])
d_a = dict(zip(df_a.index.astype('str'),df_a.Group))

In [ ]:
s_sample = '20220316_MIBI'
s_date = s_sample.split('_')[0]
#22 features cd11b monocytes, granulocytes, macrophages, and natural killer cells
#cd11c high levels on most human dendritic cells, but also on monocytes, macrophages, neutrophils, and some B cells 
lls_col = [     ['Beta-catenin-c', 'Beta-catenin-n', 'CD11b', 'CD11c', 'CD138', 'CD16', 'CD20', 'CD209',
       'CD3', 'CD31', 'CD4', 'CD45', 'CD45RO', 'CD56', 'CD63', 'CD68', 'CD8','HLA-Class-1',
        'EGFR', 'FoxP3', 'HLA-DR',  'IDO', # 'H3K27me3', 'H3K9ac', 'phospho-S6','HLA',
       'Keratin17', 'Keratin6', 'Ki67', 'Lag3', 'MPO', 'p53', 'Pan-Keratin','area','eccentricity',
       'PD-L1', 'PD1', 'SMA', 'Vimentin'], #'dsDNA', oops not in new features
]

for s_type in ['Filtered']:
    df_mi = pd.read_csv(f'{s_sample}_{s_type}MeanIntensity.csv',index_col=0)
    break
    for ls_col in lls_col:
        print(len(ls_col))
        adata = sc.AnnData(df_mi.loc[:,ls_col])
        #raw
        adata.raw = adata
        #reduce dimensionality
        sc.tl.pca(adata, svd_solver='auto')
        fig,ax=plt.subplots(figsize=(3.5,5))
        sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}_{s_type}.png')
        plt.tight_layout()
        sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)
        adata.obs['slide'] = [item.split('_')[0] for item in adata.obs.index]
        adata.obs['slide'] = adata.obs.slide.replace({'t1':'1'})
        sc.pp.scale(adata, zero_center=False, max_value=20)
        s_sample = s_sample + '_s'

        #load annotation
        # mapping
        d_a = dict(zip(df_a.index.astype('str'),df_a.Group))
        adata.obs['Group'] = adata.obs.slide.map(d_a).fillna('unknown')

        #umap
        for n_neighbors in [30]: #,30
            results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.h5ad'
            if os.path.exists(results_file):
                print('loading umap')
                adata = sc.read_h5ad(results_file)
            else:
                print('calc umap')
                # calculate neighbors 
                sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
                sc.tl.umap(adata)
            #save results
            if not os.path.exists(results_file):
                adata.write(results_file)
            #color by markers
            
            figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
            title=figname.split('.png')[0].replace('_',' ')
            sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=6,save=figname)
            #color by slide
            figname = f"Umap_slide_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
            title=figname.split('.png')[0].replace('_',' ')
            fig,ax = plt.subplots(figsize=(3,3), dpi=200)
            sc.pl.umap(adata, color='slide',wspace=.25,save=figname,ax=ax,title=title)
            
            #color by group
            
            figname = f"Umap_Group_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
            title=figname.split('.png')[0].replace('_',' ')
            fig,ax = plt.subplots(figsize=(3,3), dpi=200)
            sc.pl.umap(adata, color='Group',wspace=.25,save=figname,ax=ax,title=title)
            
            break
        break
    break


In [ ]:
adata.obs.slide.unique()

In [ ]:
#screen all
for s_slide in sorted(adata.obs.slide.unique()):
    adata.obs['test'] = (adata.obs.slide == s_slide).replace({True:1,False:0})
    sc.pl.umap(adata, color='test',title=s_slide)

In [ ]:
# weird ones (fixed)
'''
s_slide = 'RS-mTMA-5_sceneE03' #corner extra
s_slide = 'RS-mTMA-5_sceneH11' # circel imaging artifact (pmyc)
s_slide = 'RS-mTMA-5_sceneI02' #cd31 bg
#Still unique
s_slide = 'RS-mTMA-5_sceneA05' #high foxp3 bg, some float
s_slide = 'RS-mTMA-5_sceneE07' #folded tissue
s_slide = 'RS-mTMA-5_sceneH06' #bright floating tissue
s_slide = 'RS-mTMA-5_sceneI01' #bright floating tissue
s_slide = 'RS-mTMA-5_sceneI11' #epcam, ecad, CK5 neg (no tumor)
s_slide = 'RS-mTMA-5_sceneI07' #dried out bottom? pmyc weird
#new ouliers
#A05, E06, E07, F03, G09, H06, I01, I07, I11
#other tissues
# A11 spleen B10 spleen?
#B6 lung #I10 lung
#d4 normal pad, d9 adj #d12 fat pad
# E6, E12 lymph node I12 lymph node
#for s_slide in adata.obs.slide.unique():
adata.obs['test'] = (adata.obs.slide == s_slide).replace({True:1,False:0})
sc.pl.umap(adata, color='test',title=s_slide)
'''

In [ ]:
s_sample

## Cluster Leiden <a name="l8"></a>

main cell types


[contents](#contents)

In [ ]:
s_sample = '20220316_MIBI_s'
s_type = 'Filtered'
s_date = s_sample.split('_')[0]
n_neighbors = 30
n_markers = 35

results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
#leiden
for resolution in [0.6,0.4, 0.5, 0.6]: #0.4,
    results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
    if not os.path.exists(results_file):
        sc.tl.leiden(adata,resolution=resolution)
        adata.write(results_file)
    else:
        print('loading')
        adata = sc.read_h5ad(results_file)
    fig,ax = plt.subplots(figsize=(3,3),dpi=200)
    figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,)
    #fig.savefig(f'./figures/{figname}')
    break

## Annotate Leiden <a name="lbar"></a>

main cell types


[contents](#contents)

In [ ]:
s_sample = '20220316_MIBI_s'
s_type = 'Filtered'
s_date = s_sample.split('_')[0]
n_neighbors = 30
n_markers = 35
resolution = 0.6 #0.7
results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
adata = sc.read_h5ad(results_file)

In [ ]:
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,)

In [ ]:
ls_col = ['CD31','Beta-catenin-c','Pan-Keratin','Vimentin','CD45']
figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
title=figname.split('.png')[0].replace('_',' ')
sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99',ncols=5,save=figname)

In [ ]:
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[df_p.CD31>.2].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='CD31')
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[df_p.CD45>1].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='CD45')
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[(df_p.loc[:,'Beta-catenin-c']>.4) | (df_p.loc[:,'Pan-Keratin']>1)].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='Beta-catenin-c and Pan-Keratin')

adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[(df_p.Vimentin>6)].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='Vimentin')

In [ ]:
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p.columns = [item.replace(' ','_').replace('-','_') for item in df_p.columns]
df_p['leiden'] = adata.obs['leiden'].astype('str')
#CD45 above 300
#CK5 above 2048 (768)
#ecad/epcam above 2000 (768)
# CD68 above 1536
#area above 318
#all 30 neigh, 7 pcs
if results_file == '20220316_MIBI_s_30neighbors_35markers_Filtered_leiden0.4.h5ad':
    tum_clust = ['5', '13','12', '9', '7', '3', '1', '6','10']
    str_clust = ['17']
    imm_clust = ['4', '2', '14','8','15','16'] 
    endo_clust = ['11']
    fb_clust= ['0']
if results_file == '20220316_MIBI_s_30neighbors_35markers_Filtered_leiden0.7.h5ad':
    tum_clust = ['8', '18','15', '2', '4', '9', '17','21','12', '6', '3', '22','13']
    str_clust = ['16','11']
    imm_clust = ['5', '1','23', '10', '19' ,'7', '20'] 
    endo_clust = ['14']
    fb_clust= ['0']
    ls_drop= []
if results_file == '20220316_MIBI_s_30neighbors_35markers_Filtered_leiden0.6.h5ad':
    print('goodone')
    #split cd4 cd8
    df_p.loc[((df_p.leiden=='2') & (df_p.CD8 > 1.5*df_p.CD4)),'leiden'] = '2b'
    #split cd4 cd20 & cd11c
    df_p.loc[((df_p.leiden=='3') & (df_p.CD20 > 2.5*df_p.CD4)),'leiden'] = '3b'
    df_p.loc[((df_p.leiden=='3') & (df_p.CD11c > df_p.CD4)),'leiden'] = '3c'
    #split 14
    df_p.loc[((df_p.leiden=='14') & (df_p.Pan_Keratin<.5)),'leiden'] = '14b'
    tum_clust = ['14', '7', '15','9', '6', '5', '19', '11','0','8','13']
    str_clust = ['21','4']
    imm_clust = ['20','3', '2', '17','18','10','16','2b','14b','3b','3c'] 
    endo_clust = ['12']
    fb_clust= ['1']
    ls_drop= ['21']


#df_p.groupby('leiden').mean().CD45.sort_values(ascending=False).index
#set(df_p.groupby('leiden').mean().CD31.sort_values(ascending=False).index) - set(tum_clust + imm_clust + endo_clust)
#set(tum_clust)

In [ ]:
#df_p[df_p.leiden=='3'].CD20.median()
#df_p[df_p.leiden=='3'].CD4.median()
#df_p['slide'] = [item.split('_')[0] for item in df_p.index]
#s_clust = '3' #11
#df_p[df_p.leiden==s_clust].groupby('slide').count().sort_values(by='CD31',ascending=False)[:5]

In [ ]:
d_named = {'3':'CD4 T cell',
'3b':'CD20 B cell',
 '3c':'Dendritic cell',
 '2b':'CD8 T cell',
 '2':'CD4 T cell',
 '17':'FoxP3 Treg',
 '7':'Basal t.',
 '15':'PD-L1+ Basal t.',
 '14':'IDO+ Basal t.',
 '14b':'Dendritic cell',
 '18':'NK cell',
 '12':'endothelial',
 '20':'CD209+ imm.', #DC-SIGN
 # artifact'21':'quiesc str',
 '9':'Prolif. t.',
 '6':'CK lo. t.',
 '5':'Myoepithelial',
 '19':'EGFR+ t.',
 '11':'CD63+ t.',
 '13':'CK lo. t.',
 '1':'FB',
 '4':'Quies. str.',
 '0':'Luminal t.',
 '8':'Basal t.',
 '10':'Macrophage',
 '16':'neutrophil'}

In [ ]:
#clustermap
#df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
b_annot=False
b_annot = True

d_replace = {}
d_replace.update(dict(zip(tum_clust,[f'epithelial' for item in tum_clust])))
d_replace.update(dict(zip(str_clust,[f'stromal' for item in str_clust])))
d_replace.update(dict(zip(imm_clust,[f'immune' for item in imm_clust])))
d_replace.update(dict(zip(endo_clust,[f'endothelial' for item in endo_clust])))
d_replace.update(dict(zip(fb_clust,[f'fibroblast' for item in fb_clust])))
df_plot = df_p[~df_p.leiden.isin(ls_drop)].groupby('leiden').mean()

df_plot.index.name = f'leiden {resolution}'
d_color = dict(zip(['endothelial', 'epithelial', 'fibroblast', 'immune', 'stromal'],sns.color_palette()[0:5]))
if b_annot:
    row_colors = df_plot.index.astype('str').map(d_replace).map(d_color)
    g = sns.clustermap(df_plot,z_score=1,figsize=(11,7),cmap='viridis',row_colors=row_colors,#method='single',
                   vmin=-2,vmax=2)
    for label in d_color.keys():
        g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                                label=label, linewidth=0)
    g.ax_row_dendrogram.legend(loc="right", ncol=1)
    g.ax_heatmap.set_ylabel('')
    marker_genes = df_plot.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
    categories_order = df_plot.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()
    labels = [f"{item}: {d_named[item]}" for item in categories_order]
    g.ax_heatmap.set_yticklabels(labels)
else:
    g = sns.clustermap(df_plot,z_score=1,figsize=(9,7),cmap='viridis',#row_colors=row_colors,#method='single',
                   vmin=-2,vmax=2)
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_leiden_{resolution}.png',dpi=200)


In [ ]:
df_p.loc[df_p.leiden.isin(tum_clust),'leidencelltype5'] = 'epithelial'
df_p.loc[df_p.leiden.isin(str_clust),'leidencelltype5'] = 'stromal'
df_p.loc[df_p.leiden.isin(imm_clust),'leidencelltype5'] = 'immune'
df_p.loc[df_p.leiden.isin(endo_clust),'leidencelltype5'] = 'endothelial'
df_p.loc[df_p.leiden.isin(fb_clust),'leidencelltype5'] = 'fibroblast'
adata.obs['leidencelltype5'] = df_p.leidencelltype5
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden__{resolution}leidencelltype5.png'
sc.pl.umap(adata, color='leidencelltype5',ax=ax,title=figname.split('.png')[0].replace('_',' '),save=figname,wspace=0.25) #
#fig.savefig(f'{codedir}/{s_date}/{s_sample}_umap_leiden_celltype_{resolution}.png',dpi=200)

In [ ]:
fig,ax=plt.subplots(figsize=(1.4,3.7),dpi=200)
figname = f"{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.png"
df_p.groupby('leiden').count().loc[categories_order[::-1]].iloc[:,1].plot(kind='barh',title='Cell Count',ax=ax,width=0.7)
fig.savefig(f'./{s_date}/barplot_{figname}')

In [ ]:
#df_p['slide'] = [item.split('_')[0] for item in df_p.index]
df_p[df_p.leiden=='15'].Pan_Keratin.median()
#df_p[df_p.leiden=='15'].Beta_catenin_c.median()
np.quantile(df_p[df_p.leiden=='2b'].CD45,.2)
np.quantile(df_p[df_p.leiden=='1'].Vimentin,.4)

### gate celltypes

In [ ]:
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[df_p.CD31>.1].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='CD31')
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[df_p.CD45>2].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='CD45')
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[df_p.Beta_catenin_c>.2].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='Bcat')
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[df_p.Pan_Keratin>.5].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='panCK')
adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[(df_p.Vimentin>1.5)].index)).replace({True:1,False:0}).values
sc.pl.umap(adata, color='test',title='Vim')
#adata.obs['test'] = pd.Series(adata.obs.index.isin(df_p[(df_p.FN>1)].index)).replace({True:1,False:0}).values
#sc.pl.umap(adata, color='test',title='FN')

In [ ]:
#manual gating
df_p['gatedcelltype5'] = np.nan
df_p.loc[((df_p.Vimentin>1.5)),'gatedcelltype5'] = 'fibroblast'
df_p.loc[((df_p.Beta_catenin_c>.2) | (df_p.Pan_Keratin>.5)),'gatedcelltype5'] = 'epithelial'
df_p.loc[df_p.CD45>2,'gatedcelltype5'] = 'immune'
df_p.loc[df_p.CD31>.1,'gatedcelltype5'] = 'endothelial'
df_p.gatedcelltype5 = df_p.gatedcelltype5.fillna('stromal')
adata.obs['gatedcelltype5'] = df_p.gatedcelltype5
fig,ax = plt.subplots(figsize=(3,2),dpi=200)
figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}_{s_type}_gatingcelltype5.png'
sc.pl.umap(adata, color='gatedcelltype5',ax=ax,save=figname,title=figname.split('.png')[0].replace('_',' '))

In [ ]:
import sklearn
labels = sorted(df_p.leidencelltype5.fillna('stromal').unique())
confusion_matrix = sklearn.metrics.confusion_matrix(df_p.gatedcelltype5,df_p.leidencelltype5.fillna('stromal'),labels=labels)
#accuracy
i_acc = sklearn.metrics.accuracy_score(df_p.gatedcelltype5,df_p.leidencelltype5.fillna('stromal'))
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix,display_labels=labels)
fig, ax = plt.subplots(figsize = (5,5))
disp.plot(xticks_rotation='vertical',ax=ax) 
ax.set_ylabel('Gating Celltype')
ax.set_xlabel(' Clustering Celltype')
ax.set_title(f'Gating vs Clustering \n accuracy = {i_acc:.2}')
plt.tight_layout()
fig.savefig(f'{codedir}/{s_date}/Confusion_matrix_leiden{resolution}_5cells.png',dpi=300)
#4


In [ ]:
df_p.groupby('gatedcelltype5').count().CD31/len(df_p)

In [ ]:
df_p.groupby('leidencelltype5').count().CD31/len(df_p)

In [ ]:
df_p['slide'] = [item.split('_')[0] for item in df_p.index]
df_p

In [ ]:
s_out = f'{s_sample}_LeidenClusteringGating_neighbors{n_neighbors}_resolution{resolution}_markers{n_markers}_{s_type}.csv'
df_out = df_p.merge(df_mi.loc[:,['DAPI_X','DAPI_Y']],left_index=True,right_index=True)

In [ ]:
if not os.path.exists(s_out):
    print('saving csv')
    df_out.to_csv(s_out)

In [ ]:
s_out

In [ ]:
pwd

# Visualize  <a name="viz"></a>
 

[contents](#contents)

In [ ]:
from mplex_image import analyze
df_p = pd.read_csv(f'{s_sample}_LeidenClusteringGating_neighbors{n_neighbors}_resolution{resolution}_markers{n_markers}_{s_type}.csv',index_col=0)
df_p['slide_scene'] = [item.split('_cell')[0] for item in df_p.index]
df_p['slide'] = [item.split('_')[0] for item in df_p.index]

In [ ]:
s_clust = '8'#11
df_p[df_p.leiden==s_clust].groupby('slide').count().sort_values(by='CD31',ascending=False)[:5]

In [ ]:
all_colors = mpl.cm.Set1.colors + mpl.cm.Set2.colors+ mpl.cm.Set3.colors
#plot all groups spatially - leiden 
df_pos = analyze.celltype_to_bool(df_p,'leiden')
df_pos.columns = [str(item) for item in df_pos.columns]
for s_slide in sorted(set(df_p.slide)):
    s_slide= '21'
    fig,ax = plt.subplots(figsize=(5,3.5),dpi=200)
    #plot negative cells
    df_scene = df_p[(df_p.slide==s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(df_pos.columns):
        s_color=s_clust
        s_color = str(s_color)
        if len(df_p[(df_p.slide==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ax.scatter(data=df_p[(df_p.slide==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                                                                        label=f'{s_color}',s=0.1,color=all_colors[idxs])
        break
    ax.set_title(f"{s_slide}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    #ax.set_xticklabels('')
    #ax.set_yticklabels('')
    #break
    plt.legend(markerscale=10,framealpha=.5,ncol=2,bbox_to_anchor=(.8,.95)) 
    #fig.savefig(f'{codedir}/figures/{s_slide}_leiden{resolution}_scatterplot.png')
    break

In [ ]:
#plot all groups spatially  - manual
df_pos = analyze.celltype_to_bool(df_p,'leiden_celltype')
df_pos.columns = [str(item) for item in df_pos.columns]
for s_slide in sorted(set(df_p.slide)):
    fig,ax = plt.subplots(figsize=(5,3.5),dpi=200)
    s_slide= '26'#'6'#'3'
    #plot negative cells
    df_scene =df_p[(df_p.slide==s_slide)]
    ax.scatter(data=df_scene,x='DAPI_X',y='DAPI_Y',color='silver',s=0.1,label=f'')
    #for idxs, s_color_int in enumerate(range(len(df_pos.columns))):
    for idxs, s_color in enumerate(df_pos.columns):
        #print(s_color)
        #s_color = str(s_color_int)
        if len(df_p[(df_p.slide==s_slide) & (df_pos.loc[:,s_color])])>=1:
            #plot positive cells
            ax.scatter(data=df_p[(df_p.slide==s_slide) & (df_pos.loc[:,s_color])],x='DAPI_X',y='DAPI_Y',
                                                                        label=f'{s_color}',s=0.1,color=mpl.cm.tab10.colors[idxs])
        #break
    ax.set_title(f"{s_slide}", fontsize=16)
    ax.axis('equal')
    ax.set_ylim(ax.get_ylim()[::-1])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    for spines in ['left','right','top','bottom']:
        ax.spines[spines].set_visible(False)

    #break
    plt.legend(markerscale=10,framealpha=.1,bbox_to_anchor=(.85,.5)) 
    fig.savefig(f'{codedir}/figures/{s_slide}_gated_cells_scatterplot.png')
    break

In [ ]:
#old clusters (based on the )
'''
d_group = {'10': 'stroma_rich', '11': 'stroma_rich', '12': 'stroma_rich', '13': 'stroma_rich', '14': 'stroma_poor',
 '15': 'stroma_poor', '16': 'stroma_rich', '17': 'stroma_rich', '18': 'stroma_poor', '19': 'stroma_rich', '2': 'stroma_rich',
 '20': 'stroma_rich', '21': 'stroma_poor', '23': 'stroma_poor', '24': 'stroma_poor', '25': 'stroma_rich',
 '26': 'stroma_poor', '27': 'stroma_poor', '28': 'stroma_rich', '29': 'stroma_rich', '3': 'stroma_rich',
 '31': 'stroma_poor', '32': 'stroma_rich', '33': 'stroma_rich', '34': 'stroma_rich', '35': 'stroma_rich', '36': 'stroma_rich',
 '37': 'stroma_rich', '39': 'stroma_rich', '4': 'stroma_rich', '40': 'stroma_rich', '41': 'stroma_rich', '5': 'stroma_rich',
 '6': 'stroma_rich', '7': 'stroma_poor', '8': 'stroma_rich', '9': 'stroma_rich'}
d_group3 = {'10': 'imm', '11': 'str', '12': 'imm', '13': 'imm', '14': 'epi', '15': 'epi', '16': 'imm', '17': 'imm', '18': 'epi',
 '19': 'str', '2': 'imm', '20': 'str', '21': 'epi', '23': 'epi', '24': 'epi', '25': 'str', '26': 'epi', '27': 'epi', '28': 'imm',
 '29': 'imm', '3': 'imm', '31': 'epi', '32': 'imm', '33': 'str', '34': 'str', '35': 'imm', '36': 'imm', '37': 'imm', '39': 'imm',
 '4': 'imm', '40': 'imm', '41': 'imm', '5': 'imm', '6': 'str', '7': 'epi', '8': 'str', '9': 'imm'}

#new
d_group = {'1': 'stroma_rich', '10': 'stroma_rich', '11': 'stroma_rich', '12': 'stroma_rich', '13': 'stroma_rich',
 '14': 'epithelial_rich', '15': 'epithelial_rich', '16': 'stroma_rich', '17': 'stroma_rich', '18': 'epithelial_rich',
 '19': 'epithelial_rich', '2': 'epithelial_rich', '20': 'stroma_rich', '21': 'epithelial_rich',
 '22': 'epithelial_rich', '23': 'epithelial_rich', '24': 'epithelial_rich', '25': 'epithelial_rich',
 '26': 'epithelial_rich', '27': 'epithelial_rich', '28': 'stroma_rich', '29': 'stroma_rich', '3': 'stroma_rich',
 '30': 'stroma_rich', '31': 'epithelial_rich', '32': 'stroma_rich', '33': 'epithelial_rich', '34': 'stroma_rich',
 '35': 'stroma_rich', '36': 'stroma_rich', '37': 'stroma_rich', '38': 'epithelial_rich', '39': 'stroma_rich',
 '4': 'stroma_rich', '40': 'stroma_rich', '41': 'stroma_rich', '5': 'stroma_rich', '6': 'stroma_rich',
 '7': 'epithelial_rich', '8': 'epithelial_rich', '9': 'stroma_rich'}
#

df_p['Group'] = df_p.slide.map(d_group)
se_plot = df_p[df_p.Group!='not_tumor'].groupby(['leiden','Group']).count().loc[categories_order[::-1]].iloc[:,1]
se_plot_frac = se_plot/df_p[df_p.Group!='not_tumor'].groupby(['Group']).count().iloc[:,1]
fig,ax=plt.subplots(figsize=(2.5,5),dpi=200)
sns.barplot(data=se_plot_frac.reset_index(),y='leiden',x='Beta_catenin_n',hue='Group',orient='h',order=categories_order,ax=ax,hue_order = ['stroma_rich','epithelial_rich'])
ax.set_xlabel('Fraction of Cells')
ax.legend(bbox_to_anchor=(0.3,1.01))
fig.savefig(f'./{s_date}/barplot_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}_bygroup.png')
#
df_p['slide'] = [item.split('_')[0] for item in df_p.index]
df_p['Group'] = df_p.slide.map(d_group3)
se_plot = df_p[df_p.Group!='not_tumor'].groupby(['leiden','Group']).count().loc[categories_order[::-1]].iloc[:,1]
se_plot_frac = se_plot/df_p[df_p.Group!='not_tumor'].groupby(['Group']).count().iloc[:,1]
fig,ax=plt.subplots(figsize=(2.5,5),dpi=200)
sns.barplot(data=se_plot_frac.reset_index(),y='leiden',x='Beta_catenin_n',hue='Group',orient='h',order=categories_order,ax=ax)
ax.set_xlabel('Fraction of Cells')
fig.savefig(f'./{s_date}/barplot_{n_neighbors}neighbors_{len(marker_genes)}markers_leiden{resolution}_bygroup3.png')
'''

# subtyping <a name="subtype"></a>
 

[contents](#contents)

In [ ]:
s_sample = '20211109_MIBI'
resolution = 0.6
i_cells = '' #'5' # 
df_p = pd.read_csv(f'results_{s_sample}_leiden{resolution}_gated_celltype{i_cells}.csv',index_col=0)

In [ ]:
df_p['slide'] = [item.split('_')[0] for item in df_p.index]

In [ ]:
s_type = 'gated_celltype' #'leiden' #
df = df_p.loc[:,['CD31',s_type,'slide']].groupby([s_type,'slide']).CD31.count()/df_p.loc[:,['CD31','slide']].groupby(['slide']).CD31.count()
df_prop = df.unstack().T.fillna(0)
ls_col = df_prop.columns.tolist()
adata = sc.AnnData(df_prop)
adata.raw = adata
#reduce dimensionality
sc.tl.pca(adata, svd_solver='auto')
fig,ax=plt.subplots(figsize=(3.5,5))
sc.pl.highest_expr_genes(adata, n_top=48,ax=ax,save=f'Expression_{len(ls_col)}_{s_type}.png')
plt.tight_layout()
sc.pl.pca_variance_ratio(adata,n_pcs=32, log=True)

#umap
for n_neighbors in [5]: #,30
    results_file = f'{s_date}/{s_sample}_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.h5ad'
    print('calc umap')
    # calculate neighbors 
    sc.pp.neighbors(adata, n_neighbors=n_neighbors) #, method='rapids'
    sc.tl.umap(adata)

    #color by markers
    
    figname = f"Umap_markers_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    sc.pl.umap(adata, color=ls_col,vmin='p1.5',vmax='p99.5',ncols=4,save=figname,size=250)
    #survival
    '''
    adata.obs['slide'] = adata.obs.index.tolist()
    adata.obs['Group'] = adata.obs.slide.map(d_a).fillna('unknown')
    #adata.obs['Group'] = adata.obs.index.map(d_a).fillna('unknown')
            
    figname = f"Umap_Group_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    sc.pl.umap(adata, color='Group',wspace=.25,save=figname,ax=ax,title=title,size=50)
    
    # time
    figname = f"Umap_survival_{n_neighbors}neighbors_{len(ls_col)}markers_{s_type}.png"
    title=figname.split('.png')[0].replace('_',' ')
    fig,ax = plt.subplots(figsize=(3,2), dpi=200)
    adata.obs['survival'] =adata.obs.slide.map(dict(zip(df_a.index.astype('str'),df_a.Survival_time)))
    sc.pl.umap(adata, color='survival',size=50,ax=ax,title=title,save=figname)
    '''
    #save results
    if not os.path.exists(results_file):
        adata.write(results_file)
    break

In [ ]:
#leiden
for resolution in [0.2]: #0.3,
    results_file = f'{s_sample}_{n_neighbors}neighbors_{n_markers}markers_{s_type}_leiden{resolution}.h5ad'
    sc.tl.leiden(adata,resolution=resolution)
    adata.write(results_file)
    fig,ax = plt.subplots(figsize=(3,3),dpi=200)
    figname=f'leiden_{n_neighbors}_{n_markers}_{resolution}.png'
    sc.pl.umap(adata, color='leiden',ax=ax,title=figname.split('.png')[0].replace('_',' '),wspace=.25,save=figname,size=40)
    break

In [ ]:
#sns.clustermap(df_prop)

d_a = dict(zip(adata.obs.index,adata.obs.leiden))
d_color = dict(zip(adata.obs.leiden.unique(),sns.color_palette()[0:len(adata.obs.leiden.unique())]))
row_colors = df_prop.index.map(d_a).map(d_color)
g = sns.clustermap(df_prop,figsize=(5,15),cmap='viridis',
        row_colors=row_colors)
for label in d_color.keys():
    g.ax_row_dendrogram.bar(0, 0, color=d_color[label],
                        label=label, linewidth=0)
g.ax_row_dendrogram.legend(loc="right", ncol=1)
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_Group.png',dpi=200)
marker_genes = df_prop.iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_prop.iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
if resolution ==0.2:
    ls_epi = ['15', '22', '18', '24', '7', '14', '21', '23', '27', '26', '31',]#1
    ls_str = ['38', '11', '20', '34', '6', '25', '33', '19', '8',] #2
    ls_imm = [ '32', '35', '16', '28', '2', '36', 't1', '3', '5', '12', '9', '17', '39', '29', '37', '41', '10', '40', '13', '4'] #0
    ls_myl = []
elif resolution == 0.7:
    ls_epi = ['15', '22', '18', '24', '7', '14', '21', '23', '27', '26', '31',]

    ls_str = [ '38', '11', '20', '34',
     '6', '25', '33', '19', '8','32', '35', '16', '28', '2', '36', '1', '3', '5', '12', '9', '17', '39', '29', '37',
     '41', '10', '40', '13', '4']
    ls_imm = ['10', '12', '13', '16', '17', '2', '28', '29', '3', '32', '35', '36',
           '37', '39', '4', '40', '41', '5', '9', 't1']
    ls_myl = []
    ls_lym = []
elif i_Cells==5:
    ls_epi =  ['15', '18', '22', '24','14', '21', '23', '26', '27', '31', '7']
    ls_myl = ['12', '17', '3', '39', '5', '9', 't1']
    ls_lym = ['16', '28', '32', '35']
    ls_str = ['10', '13', '2', '29', '36', '37', '4', '40', '41','11', '19', '20', '25', '33', '34', '38', '6', '8',]

In [ ]:
#clustermap
df_p = pd.DataFrame(data=adata.raw.X,index=adata.obs.index,columns=adata.var.index)
df_p['leiden'] = adata.obs['leiden']
d_replace = {}
df_plot = df_p.groupby('leiden').mean()
df_plot.index.name = f'leiden {resolution}'
g = sns.clustermap(df_plot,z_score=1,figsize=(4,3.5),cmap='viridis',
                   vmin=-1.5,vmax=1.5)
#g.fig.suptitle(f'leiden {resolution}',x=.9) 
g.savefig(f'{codedir}/{s_date}/{s_sample}_clustermap_{s_type}_subtype_{resolution}.png',dpi=200)
marker_genes = df_p.groupby('leiden').mean().iloc[:,g.dendrogram_col.reordered_ind].columns.tolist()
categories_order = df_p.groupby('leiden').mean().iloc[g.dendrogram_row.reordered_ind,:].index.tolist()

In [ ]:
fig,ax=plt.subplots(figsize=(2.5,3),dpi=200)
df_p.groupby('leiden').count().loc[categories_order[::-1]].iloc[:,1].plot(kind='barh',title='Patient Count',ax=ax)
plt.tight_layout()
fig.savefig(f'./{s_date}/barplot_{s_sample}_clustermap_leiden_subtype_{resolution}.png')

In [ ]:
df_p[df_p.leiden=='0'].index

In [ ]:
categories_order


# survival <a name="surv"></a>
 

[contents](#contents)

In [ ]:
import lifelines
from lifelines import KaplanMeierFitter
from lifelines.statistics import multivariate_logrank_test
from lifelines import CoxPHFitter

In [ ]:
#
df_a.index = df_a.index.astype('str')

In [ ]:
df = df_p.merge(df_a.loc[:,['Survival','Survival_time']],left_index=True,right_index=True)

In [ ]:
T = df['Survival_time']     ## time to event
E = df['Survival']      ## event occurred or censored
groups = df.loc[:,'leiden']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in sorted(groups.unique()):
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{s_date}/KM_{resolution}_{s_type}.png',dpi=200)
   

In [ ]:
df.loc[df.index.isin(ls_epi),'Group'] ='stroma_poor'
df.loc[df.index.isin(ls_str),'Group'] ='stroma_rich'
if len(ls_imm) > 0:
    df.loc[df.index.isin(ls_imm),'Group'] ='stroma_rich'    
if len(ls_myl) > 0:
    df.loc[df.index.isin(ls_myl),'Group'] ='myl'
    df.loc[df.index.isin(ls_lym),'Group'] ='lym'
    
    

s_type = 'Group'
groups = df.loc[:,'Group']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in sorted(groups.unique()):
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{s_date}/KM_{resolution}_{s_type}.png',dpi=200)

In [ ]:
#dict(zip(df.index,df.Group3))

In [ ]:
df.loc[df.index.isin(ls_epi),'Group3'] ='epi'
df.loc[df.index.isin(ls_str),'Group3'] ='str'
df.loc[df.index.isin(ls_imm),'Group3'] ='imm'
if len(ls_myl) > 0:
    df.loc[df.index.isin(ls_myl),'Group3'] ='myl'
    df.loc[df.index.isin(ls_lym),'Group3'] ='lym'
    
    

s_type = 'Group3'
groups = df.loc[:,'Group3']  
kmf1 = KaplanMeierFitter() ## instantiate the class to create an object
fig, ax = plt.subplots(figsize=(3,3),dpi=200)
for s_group in sorted(groups.unique()):
    i1 = (groups == s_group)
    kmf1.fit(T[i1], E[i1], label=s_group)    ## fit thedata
    kmf1.plot(ax=ax,ci_show=False)
    #print(kmf1.median_survival_time_)
results = multivariate_logrank_test(event_durations=T, groups=groups, event_observed=E)
ax.set_title(f'{s_type} {resolution} \n p={results.summary.p[0]:.2}')
ax.legend(loc='upper right')
plt.tight_layout()
fig.savefig(f'{s_date}/KM_{resolution}_{s_type}.png',dpi=200)

In [ ]:
cph = CoxPHFitter()   ## Instantiate the class to create a cph object
cph.fit(df.drop(['leiden','Group','immune','stromal','endothelial'],axis=1), 'Survival_time', event_col='Survival')   ## Fit the data to train the model
#cph.print_summary()    ## HAve a look at the significance of the features
fig, ax = plt.subplots()
cph.plot(ax=ax)
ax.set_title('Cox Proportional Hazards')
plt.tight_layout()
fig.savefig('Cox_PH_epithelial.png',dpi=200)

In [ ]:
#
df['es_ratio'] = df.loc[:,'epithelial']/df.loc[:,['endothelial','immune','stromal']].sum(axis=1)
cph = CoxPHFitter()  
cph.fit(df.drop(['leiden','Group','immune','stromal','endothelial','epithelial'],axis=1), 'Survival_time', event_col='Survival')   ## Fit the data to train the model
#cph.print_summary()    ## HAve a look at the significance of the features
fig, ax = plt.subplots()
cph.plot(ax=ax)
ax.set_title('Cox Proportional Hazards')
plt.tight_layout()
fig.savefig('Cox_PH_es_ratio.png',dpi=200)

In [ ]:
#single var